In [1]:
import numpy as np
import pandas as pd
import os
import cv2
import tensorflow as tf
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight

2025-06-12 07:01:43.963734: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749711704.196408      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749711704.270499      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
IMG_SIZE = (128, 128)
BATCH_SIZE = 32
CLASSES = 43
DATA_PATH = '/kaggle/input/gtsrb-german-traffic-sign'

In [3]:
# Data Loading 
def load_data():
    # Training data
    train_df = pd.DataFrame(columns=['path', 'label'])
    for class_id in range(CLASSES):
        class_dir = os.path.join(DATA_PATH, 'train', str(class_id))
        image_paths = [os.path.join(class_dir, f) for f in os.listdir(class_dir)]
        temp_df = pd.DataFrame({'path': image_paths, 'label': class_id})
        train_df = pd.concat([train_df, temp_df], ignore_index=True)

    # Test data
    test_df = pd.read_csv(os.path.join(DATA_PATH, 'Test.csv'))
    test_df['path'] = test_df['Path'].apply(lambda x: os.path.join(DATA_PATH, x))
    test_df['label'] = test_df['ClassId'].astype(np.int32)

    # Print dataset statistics
    
    print(f"Total training images loaded: {len(train_df)}")
    print(f"Total test images loaded: {len(test_df)}")
    print(f"Number of unique classes found: {train_df['label'].nunique()}")
    
    # Stratified split
    train_df, val_df = train_test_split(
    train_df, test_size=0.15, stratify=train_df['label'], random_state=42
    )

    # Force labels to int32 in all splits
    train_df['label'] = train_df['label'].astype(int).astype('int32')
    val_df['label'] = val_df['label'].astype(int).astype('int32')
    test_df['label'] = test_df['label'].astype(int).astype('int32')

    
    print(f"\nAfter stratified split:")
    print(f"- Training samples: {len(train_df)}")
    print(f"- Validation samples: {len(val_df)} \n")
    
    
    return train_df, val_df, test_df


In [4]:
# CLAHE Preprocessing
def clahe_preprocessing(img):
    img = np.array(img, dtype=np.uint8)
    lab = cv2.cvtColor(img, cv2.COLOR_RGB2LAB)
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
    lab[:,:,0] = clahe.apply(lab[:,:,0])
    return cv2.cvtColor(lab, cv2.COLOR_LAB2RGB)

In [5]:
# Data Augmentation
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=lambda x: clahe_preprocessing(x).astype(np.float32)/255.0,
    rotation_range=25,
    zoom_range=0.2,
    width_shift_range=0.15,
    height_shift_range=0.15,
    shear_range=0.15,
    brightness_range=[0.7, 1.3],
    channel_shift_range=50,
    fill_mode='constant',
    cval=0.0
)

val_test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=lambda x: clahe_preprocessing(x).astype(np.float32)/255.0
)


In [6]:
def create_generators(train_df, val_df, test_df):
    train_gen = train_datagen.flow_from_dataframe(
        train_df,
        x_col='path',
        y_col='label',
        target_size=IMG_SIZE,
        class_mode='raw',
        batch_size=BATCH_SIZE,
        shuffle=True
    )
    
    val_gen = val_test_datagen.flow_from_dataframe(
        val_df,
        x_col='path',
        y_col='label',
        target_size=IMG_SIZE,
        class_mode='raw',
        batch_size=BATCH_SIZE,
        shuffle=False
    )
    
    test_gen = val_test_datagen.flow_from_dataframe(
        test_df,
        x_col='path',
        y_col='label',
        target_size=IMG_SIZE,
        class_mode='raw',
        batch_size=BATCH_SIZE,
        shuffle=False
    )
    
    return train_gen, val_gen, test_gen

In [7]:
from tensorflow.keras.regularizers import l2

from tensorflow.keras.layers import (
    Conv2D, BatchNormalization, MaxPool2D, Dense, 
    GlobalAveragePooling2D, Dropout, Multiply,
    Reshape, Input, Activation
)


# Attention Model 1(Channel)
class ChannelAttention(tf.keras.layers.Layer):
    def __init__(self, ratio=8):
        super().__init__()
        self.ratio = ratio
        
    def build(self, input_shape):
        self.channels = input_shape[-1]
        self.gap = GlobalAveragePooling2D()
        self.dense1 = Dense(self.channels // self.ratio, 
                          activation='relu', 
                          kernel_initializer='he_normal',
                          use_bias=False)
        self.dense2 = Dense(self.channels, 
                          activation='sigmoid',
                          kernel_initializer='he_normal',
                          use_bias=False)
        super().build(input_shape)
        
    def call(self, inputs):
        gap = self.gap(inputs) 
        gap = Reshape((1, 1, self.channels))(gap) 
        
        x = self.dense1(gap)  
        x = self.dense2(x)    
        
        return Multiply()([inputs, x]) 




# Attention Model 2(Spatial)
class SpatialAttention(tf.keras.layers.Layer):
    def __init__(self):
        super().__init__()
        self.conv = Conv2D(1, 7, padding='same', 
                          activation='sigmoid',
                          kernel_initializer='he_normal')
        
    def call(self, inputs):
        avg = tf.reduce_mean(inputs, axis=-1, keepdims=True)
        mx = tf.reduce_max(inputs, axis=-1, keepdims=True)
        x = tf.concat([avg, mx], axis=-1)
        return inputs * self.conv(x)

In [8]:
# Building CNN

def conv_block(x, filters, kernel=3, stride=1, use_attn=True):
    x = Conv2D(filters, kernel, strides=stride, padding='same',
              kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = Activation('swish')(x)
    if use_attn:
        x = ChannelAttention(ratio=8)(x)
        x = SpatialAttention()(x)
    return x


def build_model():
    inputs = Input(shape=(*IMG_SIZE, 3))
    
    # Stem
    x = conv_block(inputs, 32, stride=2, use_attn=False)
    
    # Block 1
    x = conv_block(x, 64)
    x = MaxPool2D(2)(x)
    x = Dropout(0.2)(x)
    
    # Block 2
    x = conv_block(x, 128)
    x = conv_block(x, 128)
    x = MaxPool2D(2)(x)
    x = Dropout(0.3)(x)
    
    # Block 3
    x = conv_block(x, 256)
    x = conv_block(x, 512)
    x = MaxPool2D(2)(x)
    x = Dropout(0.4)(x)
    
    # Head
    x = GlobalAveragePooling2D()(x)
    x = Dense(512, activation='swish', kernel_regularizer=l2(1e-4))(x)
    x = Dropout(0.5)(x)
    outputs = Dense(CLASSES, activation='softmax')(x)
    
    return tf.keras.Model(inputs, outputs)

In [10]:
# Load data
train_df, val_df, test_df = load_data()
train_gen, val_gen, test_gen = create_generators(train_df, val_df, test_df)

Total training images loaded: 39209
Total test images loaded: 12630
Number of unique classes found: 43

After stratified split:
- Training samples: 33327
- Validation samples: 5882 

Found 33327 validated image filenames.
Found 5882 validated image filenames.
Found 12630 validated image filenames.


In [11]:
# Class weights
class_weights = class_weight.compute_class_weight(
    'balanced', classes=np.unique(train_df['label']), y=train_df['label']
)
class_weights = {i: class_weights[i] for i in range(CLASSES)}

In [12]:
from tensorflow.keras.optimizers import AdamW

# Model & callbacks
model = build_model()
model.compile(
    optimizer=AdamW(learning_rate=1e-3, weight_decay=1e-4),
    loss='sparse_categorical_crossentropy', metrics=['accuracy']
)

callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=12, restore_best_weights=True,
                                    monitor='val_accuracy', mode='max'),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5,
                                        patience=5, min_lr=1e-6),
    tf.keras.callbacks.ModelCheckpoint('best_model.h5', save_best_only=True,
                                      monitor='val_accuracy')
]

I0000 00:00:1749712100.661858      35 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1749712100.662545      35 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


In [13]:
# Train model
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=50,
    callbacks=callbacks,
    class_weight=class_weights
)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50


I0000 00:00:1749712117.928790     108 service.cc:148] XLA service 0x7936d000bb40 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1749712117.929780     108 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1749712117.929800     108 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1749712119.307556     108 cuda_dnn.cc:529] Loaded cuDNN version 90300


   2/1042 ━━━━━━━━━━━━━━━━━━━━ 56s 55ms/step - accuracy: 0.0547 - loss: 4.1866   

I0000 00:00:1749712132.317041     108 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1042/1042 ━━━━━━━━━━━━━━━━━━━━ 406s 364ms/step - accuracy: 0.0828 - loss: 3.2913 - val_accuracy: 0.3245 - val_loss: 2.0553 - learning_rate: 0.0010
Epoch 2/50
1042/1042 ━━━━━━━━━━━━━━━━━━━━ 223s 214ms/step - accuracy: 0.4226 - loss: 1.7107 - val_accuracy: 0.8477 - val_loss: 0.5676 - learning_rate: 0.0010
Epoch 3/50
1042/1042 ━━━━━━━━━━━━━━━━━━━━ 224s 215ms/step - accuracy: 0.7507 - loss: 0.7692 - val_accuracy: 0.9442 - val_loss: 0.2680 - learning_rate: 0.0010
Epoch 4/50
1042/1042 ━━━━━━━━━━━━━━━━━━━━ 223s 214ms/step - accuracy: 0.8561 - loss: 0.5055 - val_accuracy: 0.9638 - val_loss: 0.1875 - learning_rate: 0.0010
Epoch 5/50
1042/1042 ━━━━━━━━━━━━━━━━━━━━ 220s 211ms/step - accuracy: 0.8934 - loss: 0.3773 - val_accuracy: 0.9740 - val_loss: 0.1579 - learning_rate: 0.0010
Epoch 6/50
1042/1042 ━━━━━━━━━━━━━━━━━━━━ 226s 217ms/step - accuracy: 0.9075 - loss: 0.3326 - val_accuracy: 0.9840 - val_loss: 0.1146 - learning_rate: 0.0010
Epoch 7/50
1042/1042 ━━━━━━━━━━━━━━━━━━━━ 221s 212ms/step - acc

In [18]:
model.load_weights('best_model.h5')

# Basic evaluation
test_loss, test_acc = model.evaluate(test_gen)
print(f"\nBase Test Accuracy: {test_acc*100:.2f}%")

395/395 ━━━━━━━━━━━━━━━━━━━━ 29s 73ms/step - accuracy: 0.9881 - loss: 0.0595

Base Test Accuracy: 98.80%
